[View in Colaboratory](https://colab.research.google.com/github/averma12/Beginning_Data_Science/blob/master/Word_Embeddings.ipynb)

In [0]:
import numpy as np

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import sys
reviews = '17HG0xJhKWH0lssnmWbBFeDlRaux16And'
labels  = '165f4yUxt8PPVpuRgItVDEGPB2mmYS-MF'
reviews_downloaded = drive.CreateFile({'id':reviews})
labels_downloaded  = drive.CreateFile({'id':labels})
reviews_downloaded.GetContentFile('reviews.txt')
labels_downloaded.GetContentFile('labels.txt')

In [0]:
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()
len(raw_reviews)

25000

In [0]:
raw_reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \n'

In [0]:
f = open('labels.txt')
raw_labels = f.readlines()
f.close()
len(raw_labels)

25000

In [0]:
raw_labels[0]

'positive\n'

In [0]:
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [0]:
# tokens is a list of lists which contains all the words.

In [0]:
# build a vocab
vocab = set()
for sentence in tokens:
  for word in sentence:
    if(len(word)>0):
      vocab.add(word)
vocab = list(vocab)

In [0]:
len(vocab),vocab[:100]

(74074,
 ['dream',
  'define',
  'utilises',
  'ddt',
  'saddening',
  'doth',
  'resplendent',
  'kramp',
  'vllad',
  'inclines',
  'kenan',
  'tended',
  'acceptence',
  'normed',
  'exhooker',
  'piquant',
  'plainclothes',
  'zaniness',
  'ize',
  'confidante',
  'aspect',
  'mentos',
  'maru',
  'roux',
  'qestions',
  'miserably',
  'yaayyyyy',
  'deconstructed',
  'matchup',
  'dostoyevsky',
  'cantonese',
  'insted',
  'overseeing',
  'borden',
  'nietszche',
  'model',
  'moviegoer',
  'comdie',
  'dues',
  'fozzie',
  'bohlen',
  'shuffling',
  'supervillain',
  'steams',
  'lunges',
  'cheech',
  'dissappoint',
  'geeeeeetttttttt',
  'honeymoon',
  'mesh',
  'shvollenpecker',
  'humiliating',
  'balder',
  'turandot',
  'lipstick',
  'essy',
  'multipurpose',
  'vents',
  'fixate',
  'negotiate',
  'piledriver',
  'ounces',
  'devoutly',
  'protocols',
  'overrated',
  'testicles',
  'tobe',
  'inglorious',
  'ideologue',
  'fadeout',
  'compound',
  'flour',
  'recopies',


In [0]:
# Build a lookup table of words
# word2index and index2word
word2index = {}
index2word = {}
for index,word in enumerate(vocab):
  word2index[word] = index
  index2word[index] = word

In [0]:
word2index['beautiful'],index2word[3049]

(3049, 'beautiful')

In [0]:
input_dataset = list()
for sentence in tokens:
  sent_indices = list()
  for word in sentence:
    try:
      
      sent_indices.append(word2index[word])
    except:
      ""
  input_dataset.append(sent_indices)

In [0]:
input_dataset[0]
#input dataset is a list of all the sentences . The sentences have been encoded with their dictionary index values

[12997,
 6643,
 48096,
 61721,
 67145,
 36106,
 44860,
 44861,
 46933,
 2377,
 44574,
 62036,
 58529,
 62042,
 23758,
 52849,
 9300,
 72950,
 36459,
 27193,
 46395,
 37940,
 55176,
 15705,
 649,
 60628,
 48167,
 50231,
 72978,
 26940,
 63829,
 985,
 33946,
 9340,
 44384,
 6195,
 9629,
 30456,
 69791,
 22936,
 46452,
 31627,
 37991,
 9656,
 8218,
 71574,
 53232,
 27821,
 139,
 63014,
 14012,
 1624,
 55569,
 73352,
 66998,
 43570,
 14031,
 14342,
 46799,
 14935,
 1356,
 176,
 43312,
 49758,
 30229,
 27028,
 66723,
 32285,
 59001,
 59570,
 31426,
 61914,
 37790,
 16420,
 64285,
 71661,
 52151,
 34677,
 57891,
 60501,
 30271,
 47168,
 5760,
 60506,
 73742,
 29691,
 57636,
 63707,
 50129,
 66233,
 47758,
 54203,
 45708]

In [0]:
target_dataset = list()
for label in raw_labels:
  if label == 'positive\n':
    target_dataset.append(1)
  else:
    target_dataset.append(0)

In [0]:
np.random.seed(1)
def sigmoid(x):
  return 1/(1+np.exp(-x))
alpha,iterations = (0.01,4)
hidden = 100
weights_0_1 =  0.2 * np.random.random((len(vocab),hidden)) -1
weights_1_2 =  0.2 * np.random.random((hidden,1)) -1

In [0]:
weights_1_2.shape

(100, 1)

In [0]:
correct,total = (0,0)

In [0]:
for iter in range(iterations):
  for i in range(len(input_dataset) - 1000):
    x,y = (input_dataset[i],target_dataset[i])
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    layer_2_delta = layer_2 - y
    layer_1_delta = layer_2_delta.dot(weights_1_2.T)
    weights_0_1[x] -= layer_1_delta * alpha
    weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha
    if(np.abs(layer_2_delta) < 0.5):
      
      correct += 1
    total = 1
    if(i % 10 == 9):
      progress = str(i/float(len(input_dataset)))
      #sys.stdout.write('\rIter:'+str(iter)\ +' Progress:'+progress[2:4]\ +'.'+progress[4:6]\ +'% Training Accuracy:'\ + str(correct/float(total))) + '%')
      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
  x = input_dataset[i]
  y = target_dataset[i]
  layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
  layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
  if(np.abs(layer_2 - y) < 0.5):
    correct += 1
  total += 1
  print("Test Accuracy:" + str(correct / float(total)))
  

Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.0
Test Accuracy:0.07142857142857142
Test Accuracy:0.06666666666666667
Test Accuracy:0.125
Test Accuracy:0.11764705882352941
Test Accuracy:0.1111111111111111
Test Accuracy:0.10526315789473684
Test Accuracy:0.1
Test Accuracy:0.09523809523809523
Test Accuracy:0.09090909090909091
Test Accuracy:0.08695652173913043
Test Accuracy:0.08333333333333333
Test Accuracy:0.08
Test Accuracy:0.07692307692307693
Test Accuracy:0.07407407407407407
Test Accuracy:0.07142857142857142
Test Accuracy:0.06896551724137931
Test Accuracy:0.1
Test Accuracy:0.0967741935483871
Test Accuracy:0.125
Test Accuracy:0.12121212121212122
Test Accuracy:0.14705882352941177
Test Accuracy:0.14285714285714285
Test Accuracy:0.1388888888888889
Test Accuracy:0.13513513513513514
Test Accuracy:0.131578947368

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Test Accuracy:0.07674943566591422
Test Accuracy:0.07657657657657657
Test Accuracy:0.07640449438202247
Test Accuracy:0.07623318385650224
Test Accuracy:0.07606263982102908
Test Accuracy:0.07589285714285714
Test Accuracy:0.0757238307349666
Test Accuracy:0.07555555555555556
Test Accuracy:0.07538802660753881
Test Accuracy:0.0752212389380531
Test Accuracy:0.07505518763796909
Test Accuracy:0.07488986784140969
Test Accuracy:0.07472527472527472
Test Accuracy:0.07456140350877193
Test Accuracy:0.07439824945295405
Test Accuracy:0.07423580786026202
Test Accuracy:0.07407407407407407
Test Accuracy:0.07391304347826087
Test Accuracy:0.0737527114967462
Test Accuracy:0.0735930735930736
Test Accuracy:0.0734341252699784
Test Accuracy:0.07327586206896551
Test Accuracy:0.07311827956989247
Test Accuracy:0.07296137339055794
Test Accuracy:0.0728051391862955
Test Accuracy:0.07264957264957266
Test Accuracy:0.07249466950959488
Test Accuracy:0.07234042553191489
Test Accuracy:0.07218683651804671
Test Accuracy:0.0720

In [0]:
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
  
  x = input_dataset[i]
  y = target_dataset[i] 
  layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
  layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
 
  if(np.abs(layer_2 - y) < 0.5):
    
    correct += 1
  total += 1
print("Test Accuracy:" + str(correct / float(total)))

SyntaxError: ignored

In [0]:
??weights_0_1[input_dataset[0]]

In [0]:
sigmoid(np.sum(weights_0_1[input_dataset[0]],axis=0))

array([3.32672568e-37, 4.38339166e-37, 4.49702101e-37, 2.94607975e-37,
       3.60152745e-37, 8.26410231e-37, 7.76742067e-37, 6.66631693e-37,
       1.86566400e-37, 1.41173088e-36, 3.08521057e-37, 1.91830891e-37,
       2.13002454e-37, 3.22938925e-37, 4.05310285e-37, 5.31408462e-37,
       2.14773520e-37, 7.33084379e-37, 6.79485167e-37, 4.26954602e-37,
       4.52242097e-37, 1.61736891e-37, 4.39607018e-37, 5.90526389e-37,
       6.98415206e-37, 2.90538702e-37, 3.41690956e-37, 7.47271797e-37,
       5.33780762e-37, 2.95001521e-37, 7.98197937e-37, 1.61423488e-37,
       3.25619818e-37, 2.79466260e-37, 3.71440964e-37, 2.75365745e-37,
       4.57338583e-37, 2.30328089e-37, 2.59952047e-37, 3.57448156e-37,
       1.75097258e-37, 3.33665778e-37, 7.90479291e-37, 4.49591250e-37,
       1.07596006e-36, 3.61145114e-37, 3.42707685e-37, 5.93694980e-37,
       9.08375788e-37, 3.98162079e-37, 6.09312277e-37, 4.27361824e-37,
       5.04666992e-37, 4.42278921e-37, 6.34694130e-37, 7.76841268e-37,
      